In [ ]:
!pip install -q "griptape[drivers-prompt-huggingface-pipeline]" "transformers" "accelerate" "sentencepiece"

import textwrap
from griptape.structures import Workflow, Agent
from griptape.tasks import PromptTask
from griptape.tools import CalculatorTool
from griptape.rules import Rule, Ruleset
from griptape.drivers.prompt.huggingface_pipeline import HuggingFacePipelinePromptDriver

local_driver = HuggingFacePipelinePromptDriver(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_tokens=256,
)

def show(title, content):
    print(f"\n{'='*20} {title} {'='*20}")
    print(textwrap.fill(str(content), width=100))

In [ ]:
math_agent = Agent(
    prompt_driver=local_driver,
    tools=[CalculatorTool()],
)

math_response = math_agent.run(
    "Compute (37*19)/7 and explain the steps briefly."
)

show("Agent + CalculatorTool", math_response.output.value)

In [ ]:
world_task = PromptTask(
    input="Create a vivid fictional world using these cues: {{ args[0] }}.\nDescribe geography, culture, and conflicts in 3–5 paragraphs.",
    id="world",
    prompt_driver=local_driver,
)

def character_task(task_id, name):
    return PromptTask(
        input=(
            "Based on the world below, invent a detailed character named {{ name }}.\n"
            "World description:\n{{ parent_outputs['world'] }}\n\n"
            "Describe their background, desires, flaws, and one secret."
        ),
        id=task_id,
        parent_ids=["world"],
        prompt_driver=local_driver,
        context={"name": name},
    )

scotty_task = character_task("scotty", "Scotty")
annie_task = character_task("annie", "Annie")

In [ ]:
style_ruleset = Ruleset(
    name="StoryStyle",
    rules=[
        Rule("Write in a cinematic, emotionally engaging style."),
        Rule("Avoid explicit gore or graphic violence."),
        Rule("Keep the story between 400 and 700 words."),
    ],
)

story_task = PromptTask(
    input=(
        "Write a complete short story using the following elements.\n\n"
        "World:\n{{ parent_outputs['world'] }}\n\n"
        "Character 1 (Scotty):\n{{ parent_outputs['scotty'] }}\n\n"
        "Character 2 (Annie):\n{{ parent_outputs['annie'] }}\n\n"
        "The story must have a clear beginning, middle, and end, with a meaningful character decision near the climax."
    ),
    id="story",
    parent_ids=["world", "scotty", "annie"],
    prompt_driver=local_driver,
    rulesets=[style_ruleset],
)

story_workflow = Workflow(tasks=[world_task, scotty_task, annie_task, story_task])
topic = "tidally locked ocean world with floating cities powered by storms"
story_workflow.run(topic)

In [ ]:
world_text = world_task.output.value
scotty_text = scotty_task.output.value
annie_text = annie_task.output.value
story_text = story_task.output.value

show("Generated World", world_text)
show("Character: Scotty", scotty_text)
show("Character: Annie", annie_text)
show("Final Story", story_text)

def summarize_story(text):
    paragraphs = [p for p in text.split("\n") if p.strip()]
    length = len(text.split())
    structure_score = min(len(paragraphs), 10)
    return {
        "word_count": length,
        "paragraphs": len(paragraphs),
        "structure_score_0_to_10": structure_score,
    }

metrics = summarize_story(story_text)
show("Story Metrics", metrics)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

[12/04/25 15:03:25] INFO     PromptTask a133062134554564b825e534b10e7eea                                           
                             Input: Compute (37*19)/7 and explain the steps briefly.

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]